In [17]:
import PyPDF2
import spacy
import json
from typing import List, Dict, Any
from pathlib import Path
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import logging

# Set up logging
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')

class SemanticChunker:
    def __init__(self, pdf_path: str):
        self.pdf_path = pdf_path
        logging.info(f"Initializing SemanticChunker with PDF: {pdf_path}")
        self.nlp = spacy.load("en_core_web_sm")
        self.toc = []
        
    def debug_pdf_content(self):
        """Debug function to check PDF content extraction"""
        logging.info("Running PDF content debug...")
        
        try:
            with open(self.pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                
                # Check first few pages
                for i in range(min(5, len(reader.pages))):
                    page_text = reader.pages[i].extract_text()
                    logging.info(f"\nPage {i+1} Preview (first 200 chars):")
                    logging.info("-" * 50)
                    logging.info(page_text[:200])
                    logging.info("-" * 50)
                    logging.info(f"Page {i+1} length: {len(page_text)} characters")
        
        except Exception as e:
            logging.error(f"Error in PDF debug: {str(e)}")

    def extract_toc_and_content(self):
        """Extract table of contents and content from PDF"""
        logging.info("Extracting ToC and content from PDF...")
        try:
            with open(self.pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                content_text = ""
                
                # Extract content from all pages
                for i in range(len(reader.pages)):
                    page_text = reader.pages[i].extract_text()
                    content_text += page_text + "\n\n"
                    
                logging.info(f"Extracted {len(content_text)} characters of content")
                return content_text
                
        except Exception as e:
            logging.error(f"Error reading PDF: {str(e)}")
            raise

    def create_chunks(self, content_text: str, chunk_size: int = 1000) -> List[Dict[str, Any]]:
        """Create semantic chunks from content"""
        logging.info("Creating semantic chunks...")
        
        # Split content into paragraphs
        paragraphs = [p.strip() for p in content_text.split('\n\n') if p.strip()]
        logging.info(f"Split content into {len(paragraphs)} paragraphs")
        
        if not paragraphs:
            logging.error("No paragraphs found in content")
            return []
        
        # Fit TF-IDF on all paragraphs
        vectorizer = TfidfVectorizer(
            max_features=100,
            stop_words='english',
            min_df=1
        )
        
        try:
            vectorizer.fit(paragraphs)
            logging.info("TF-IDF vectorizer fitted successfully")
        except Exception as e:
            logging.error(f"Error fitting TF-IDF vectorizer: {str(e)}")
            return []
        
        chunks = []
        current_chunk = []
        current_length = 0
        chunk_id = 1
        
        for para in paragraphs:
            para_length = len(para)
            
            if current_length + para_length > chunk_size and current_chunk:
                try:
                    # Process current chunk
                    chunk_text = '\n\n'.join(current_chunk)
                    doc = self.nlp(chunk_text)
                    
                    # Get topics
                    chunk_vector = vectorizer.transform([chunk_text])
                    feature_names = vectorizer.get_feature_names_out()
                    top_indices = chunk_vector.toarray()[0].argsort()[-5:][::-1]
                    top_terms = [feature_names[i] for i in top_indices if chunk_vector.toarray()[0][i] > 0]
                    
                    chunk_data = {
                        'id': f"chunk_{chunk_id}",
                        'content': chunk_text,
                        'topics': top_terms,
                        'tokens': len(doc),
                        'entities': [{'text': ent.text, 'label': ent.label_} 
                                   for ent in doc.ents]
                    }
                    
                    chunks.append(chunk_data)
                    logging.info(f"Created chunk {chunk_id}:")
                    logging.info(f"  Topics: {', '.join(top_terms)}")
                    logging.info(f"  Tokens: {len(doc)}")
                    
                    # Reset for next chunk
                    current_chunk = [para]
                    current_length = para_length
                    chunk_id += 1
                    
                except Exception as e:
                    logging.error(f"Error processing chunk {chunk_id}: {str(e)}")
                    current_chunk = [para]
                    current_length = para_length
                    chunk_id += 1
            else:
                current_chunk.append(para)
                current_length += para_length
        
        # Process the last chunk
        if current_chunk:
            try:
                chunk_text = '\n\n'.join(current_chunk)
                doc = self.nlp(chunk_text)
                
                chunk_vector = vectorizer.transform([chunk_text])
                feature_names = vectorizer.get_feature_names_out()
                top_indices = chunk_vector.toarray()[0].argsort()[-5:][::-1]
                top_terms = [feature_names[i] for i in top_indices if chunk_vector.toarray()[0][i] > 0]
                
                chunk_data = {
                    'id': f"chunk_{chunk_id}",
                    'content': chunk_text,
                    'topics': top_terms,
                    'tokens': len(doc)
                }
                
                chunks.append(chunk_data)
                logging.info(f"Created final chunk {chunk_id}")
                
            except Exception as e:
                logging.error(f"Error processing final chunk: {str(e)}")
        
        logging.info(f"Total chunks created: {len(chunks)}")
        return chunks

    def process_and_save(self, output_path: str):
        """Process PDF and save chunks to JSON"""
        logging.info("Starting PDF processing...")
        try:
            # Extract content
            content_text = self.extract_toc_and_content()
            if not content_text:
                logging.error("No content extracted from PDF")
                return
            
            # Create chunks
            chunks = self.create_chunks(content_text)
            
            # Save to JSON
            output_data = {
                'metadata': {
                    'source': self.pdf_path,
                    'total_chunks': len(chunks),
                    'chunking_strategy': 'semantic_with_toc'
                },
                'chunks': chunks
            }
            
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(output_data, f, indent=2, ensure_ascii=False)
            
            logging.info(f"Successfully saved {len(chunks)} chunks to {output_path}")
            
        except Exception as e:
            logging.error(f"Error in processing PDF: {str(e)}")
            raise


In [18]:
# Usage:
chunker = SemanticChunker('s3-userguide.pdf')
chunker.debug_pdf_content()  # Run this before process_and_save to check content


2025-02-07 13:42:44,265 - INFO - Initializing SemanticChunker with PDF: s3-userguide.pdf
2025-02-07 13:42:44,846 - INFO - Running PDF content debug...
2025-02-07 13:42:45,870 - INFO - 
Page 1 Preview (first 200 chars):
2025-02-07 13:42:45,871 - INFO - --------------------------------------------------
2025-02-07 13:42:45,871 - INFO - User Guide
Amazon Simple Storage Service
API Version 2006-03-01
Copyright © 2024 Amazon Web Services, Inc. and/or its aﬃliates. All rights reserved.
2025-02-07 13:42:45,872 - INFO - --------------------------------------------------
2025-02-07 13:42:45,872 - INFO - Page 1 length: 148 characters
2025-02-07 13:42:45,904 - INFO - 
Page 2 Preview (first 200 chars):
2025-02-07 13:42:45,905 - INFO - --------------------------------------------------
2025-02-07 13:42:45,906 - INFO - Amazon Simple Storage Service User Guide
Amazon Simple Storage Service: User Guide
Copyright © 2024 Amazon Web Services, Inc. and/or its aﬃliates. All rights reserved.
Amazon's tradem

In [19]:
# Then process if content looks good
chunker.process_and_save('NEW-s3-userguide-chunks.json')

2025-02-07 13:42:47,249 - INFO - Starting PDF processing...
2025-02-07 13:42:47,250 - INFO - Extracting ToC and content from PDF...
2025-02-07 13:45:10,863 - INFO - Extracted 4323539 characters of content
2025-02-07 13:45:10,864 - INFO - Creating semantic chunks...
2025-02-07 13:45:10,873 - INFO - Split content into 2500 paragraphs
2025-02-07 13:45:11,264 - INFO - TF-IDF vectorizer fitted successfully
2025-02-07 13:45:11,299 - INFO - Created chunk 1:
2025-02-07 13:45:11,300 - INFO -   Topics: amazon, service, simple, storage, user
2025-02-07 13:45:11,300 - INFO -   Tokens: 146
2025-02-07 13:45:11,360 - INFO - Created chunk 2:
2025-02-07 13:45:11,361 - INFO -   Topics: s3, amazon, access, management, aws
2025-02-07 13:45:11,361 - INFO -   Tokens: 224
2025-02-07 13:45:11,435 - INFO - Created chunk 3:
2025-02-07 13:45:11,436 - INFO -   Topics: bucket, buckets, object, access, objects
2025-02-07 13:45:11,436 - INFO -   Tokens: 262
2025-02-07 13:45:11,497 - INFO - Created chunk 4:
2025-02-0

2025-02-07 13:45:14,291 - INFO - Created chunk 45:
2025-02-07 13:45:14,292 - INFO -   Topics: bucket, delete, choose, use, objects
2025-02-07 13:45:14,293 - INFO -   Tokens: 290
2025-02-07 13:45:14,343 - INFO - Created chunk 46:
2025-02-07 13:45:14,344 - INFO -   Topics: use, s3, buckets, amazon, directory
2025-02-07 13:45:14,344 - INFO -   Tokens: 297
2025-02-07 13:45:14,434 - INFO - Created chunk 47:
2025-02-07 13:45:14,434 - INFO -   Topics: policies, access, iam, s3, acls
2025-02-07 13:45:14,435 - INFO -   Tokens: 541
2025-02-07 13:45:14,512 - INFO - Created chunk 48:
2025-02-07 13:45:14,513 - INFO -   Topics: s3, amazon, cli, use, aws
2025-02-07 13:45:14,514 - INFO -   Tokens: 500
2025-02-07 13:45:14,580 - INFO - Created chunk 49:
2025-02-07 13:45:14,581 - INFO -   Topics: aws, s3, amazon, use, operations
2025-02-07 13:45:14,582 - INFO -   Tokens: 400
2025-02-07 13:45:14,631 - INFO - Created chunk 50:
2025-02-07 13:45:14,631 - INFO -   Topics: s3, data, replication, batch, amazon


2025-02-07 13:45:17,124 - INFO -   Topics: bucket, lifecycle, objects, s3, rule
2025-02-07 13:45:17,124 - INFO -   Tokens: 475
2025-02-07 13:45:17,178 - INFO - Created chunk 92:
2025-02-07 13:45:17,178 - INFO -   Topics: bucket, command, s3, aws, cli
2025-02-07 13:45:17,179 - INFO -   Tokens: 341
2025-02-07 13:45:17,246 - INFO - Created chunk 93:
2025-02-07 13:45:17,246 - INFO -   Topics: lifecycle, bucket, rule, conﬁguration, objects
2025-02-07 13:45:17,247 - INFO -   Tokens: 463
2025-02-07 13:45:17,318 - INFO - Created chunk 94:
2025-02-07 13:45:17,319 - INFO -   Topics: bucket, delete, access, zone, points
2025-02-07 13:45:17,320 - INFO -   Tokens: 497
2025-02-07 13:45:17,374 - INFO - Created chunk 95:
2025-02-07 13:45:17,375 - INFO -   Topics: delete, bucket, iam, permissions, s3
2025-02-07 13:45:17,376 - INFO -   Tokens: 361
2025-02-07 13:45:17,423 - INFO - Created chunk 96:
2025-02-07 13:45:17,423 - INFO -   Topics: import, amazonaws, com, bucket, delete
2025-02-07 13:45:17,424 -

2025-02-07 13:45:19,709 - INFO -   Tokens: 357
2025-02-07 13:45:19,776 - INFO - Created chunk 138:
2025-02-07 13:45:19,777 - INFO -   Topics: object, value, api, upload, set
2025-02-07 13:45:19,778 - INFO -   Tokens: 423
2025-02-07 13:45:19,835 - INFO - Created chunk 139:
2025-02-07 13:45:19,836 - INFO -   Topics: amazonaws, demo, amzn, s3, key
2025-02-07 13:45:19,836 - INFO -   Tokens: 358
2025-02-07 13:45:19,914 - INFO - Created chunk 140:
2025-02-07 13:45:19,914 - INFO -   Topics: object, copy, upload, s3, want
2025-02-07 13:45:19,915 - INFO -   Tokens: 520
2025-02-07 13:45:19,974 - INFO - Created chunk 141:
2025-02-07 13:45:19,974 - INFO -   Topics: choose, destination, copy, objects, settings
2025-02-07 13:45:19,975 - INFO -   Tokens: 384
2025-02-07 13:45:20,047 - INFO - Created chunk 142:
2025-02-07 13:45:20,048 - INFO -   Topics: choose, destination, key, specify, copy
2025-02-07 13:45:20,049 - INFO -   Tokens: 454
2025-02-07 13:45:20,120 - INFO - Created chunk 143:
2025-02-07 1

2025-02-07 13:45:22,426 - INFO -   Tokens: 300
2025-02-07 13:45:22,463 - INFO - Created chunk 184:
2025-02-07 13:45:22,464 - INFO -   Topics: multipart, bucket, setting, s3, object
2025-02-07 13:45:22,465 - INFO -   Tokens: 236
2025-02-07 13:45:22,517 - INFO - Created chunk 185:
2025-02-07 13:45:22,518 - INFO -   Topics: upload, multipart, encryption, using, api
2025-02-07 13:45:22,519 - INFO -   Tokens: 336
2025-02-07 13:45:22,578 - INFO - Created chunk 186:
2025-02-07 13:45:22,579 - INFO -   Topics: import, amazonaws, com, upload, string
2025-02-07 13:45:22,580 - INFO -   Tokens: 305
2025-02-07 13:45:22,629 - INFO - Created chunk 187:
2025-02-07 13:45:22,630 - INFO -   Topics: upload, multipart, new, list, add
2025-02-07 13:45:22,631 - INFO -   Tokens: 270
2025-02-07 13:45:22,684 - INFO - Created chunk 188:
2025-02-07 13:45:22,684 - INFO -   Topics: using, multipart, upload, amazon, s3
2025-02-07 13:45:22,685 - INFO -   Tokens: 318
2025-02-07 13:45:22,734 - INFO - Created chunk 189:


...

2025-02-07 13:47:20,428 - INFO -   Tokens: 317
2025-02-07 13:47:20,486 - INFO - Created chunk 2291:
2025-02-07 13:47:20,486 - INFO -   Topics: encryption, destination, choose, key, bucket
2025-02-07 13:47:20,488 - INFO -   Tokens: 415
2025-02-07 13:47:20,541 - INFO - Created chunk 2292:
2025-02-07 13:47:20,541 - INFO -   Topics: kms, key, sse, keys, aws
2025-02-07 13:47:20,542 - INFO -   Tokens: 361
2025-02-07 13:47:20,609 - INFO - Created chunk 2293:
2025-02-07 13:47:20,609 - INFO -   Topics: object, status, information, access, multipart
2025-02-07 13:47:20,610 - INFO -   Tokens: 496
2025-02-07 13:47:20,657 - INFO - Created chunk 2294:
2025-02-07 13:47:20,658 - INFO -   Topics: destination, object, s3, list, source
2025-02-07 13:47:20,658 - INFO -   Tokens: 309
2025-02-07 13:47:20,724 - INFO - Created chunk 2295:
2025-02-07 13:47:20,725 - INFO -   Topics: source, destination, bucket, demo, amzn
2025-02-07 13:47:20,725 - INFO -   Tokens: 466
2025-02-07 13:47:20,779 - INFO - Created ch

2025-02-07 13:47:23,048 - INFO -   Tokens: 435
2025-02-07 13:47:23,106 - INFO - Created chunk 2337:
2025-02-07 13:47:23,106 - INFO -   Topics: public, access, settings, bucket, account
2025-02-07 13:47:23,107 - INFO -   Tokens: 401
2025-02-07 13:47:23,136 - INFO - Created chunk 2338:
2025-02-07 13:47:23,137 - INFO -   Topics: public, settings, access, bucket, policy
2025-02-07 13:47:23,137 - INFO -   Tokens: 185
2025-02-07 13:47:23,184 - INFO - Created chunk 2339:
2025-02-07 13:47:23,185 - INFO -   Topics: bucket, policy, choose, public, access
2025-02-07 13:47:23,186 - INFO -   Tokens: 318
2025-02-07 13:47:23,244 - INFO - Created chunk 2340:
2025-02-07 13:47:23,244 - INFO -   Topics: bucket, acls, owner, control, access
2025-02-07 13:47:23,245 - INFO -   Tokens: 430
2025-02-07 13:47:23,300 - INFO - Created chunk 2341:
2025-02-07 13:47:23,301 - INFO -   Topics: server, access, grant, choose, bucket
2025-02-07 13:47:23,301 - INFO -   Tokens: 400
2025-02-07 13:47:23,347 - INFO - Created 

2025-02-07 13:47:25,398 - INFO - Created chunk 2382:
2025-02-07 13:47:25,399 - INFO -   Topics: choose, create, policy, bucket, default
2025-02-07 13:47:25,400 - INFO -   Tokens: 393
2025-02-07 13:47:25,452 - INFO - Created chunk 2383:
2025-02-07 13:47:25,453 - INFO -   Topics: policy, bucket, choose, console, aws
2025-02-07 13:47:25,454 - INFO -   Tokens: 362
2025-02-07 13:47:25,514 - INFO - Created chunk 2384:
2025-02-07 13:47:25,515 - INFO -   Topics: bucket, console, choose, value, s3
2025-02-07 13:47:25,515 - INFO -   Tokens: 443
2025-02-07 13:47:25,568 - INFO - Created chunk 2385:
2025-02-07 13:47:25,568 - INFO -   Topics: request, add, use, choose, console
2025-02-07 13:47:25,569 - INFO -   Tokens: 365
2025-02-07 13:47:25,624 - INFO - Created chunk 2386:
2025-02-07 13:47:25,625 - INFO -   Topics: choose, status, console, request, aws
2025-02-07 13:47:25,625 - INFO -   Tokens: 393
2025-02-07 13:47:25,682 - INFO - Created chunk 2387:
2025-02-07 13:47:25,682 - INFO -   Topics: choo

2025-02-07 13:47:27,969 - INFO -   Tokens: 415
2025-02-07 13:47:28,023 - INFO - Created chunk 2428:
2025-02-07 13:47:28,024 - INFO -   Topics: delete, zone, choose, console, amazon
2025-02-07 13:47:28,025 - INFO -   Tokens: 354
2025-02-07 13:47:28,060 - INFO - Created chunk 2429:
2025-02-07 13:47:28,061 - INFO -   Topics: choose, delete, bucket, zone, set
2025-02-07 13:47:28,062 - INFO -   Tokens: 226
2025-02-07 13:47:28,101 - INFO - Created chunk 2430:
2025-02-07 13:47:28,101 - INFO -   Topics: request, aws, service, cli, default
2025-02-07 13:47:28,103 - INFO -   Tokens: 238
2025-02-07 13:47:28,142 - INFO - Created chunk 2431:
2025-02-07 13:47:28,143 - INFO -   Topics: aws, command, cli, amazon, s3
2025-02-07 13:47:28,143 - INFO -   Tokens: 253
2025-02-07 13:47:28,185 - INFO - Created chunk 2432:
2025-02-07 13:47:28,186 - INFO -   Topics: kms, keys, sse, lifecycle, s3
2025-02-07 13:47:28,187 - INFO -   Tokens: 270
2025-02-07 13:47:28,215 - INFO - Created chunk 2433:
2025-02-07 13:47:

2025-02-07 13:47:29,894 - INFO -   Tokens: 309
2025-02-07 13:47:29,921 - INFO - Created chunk 2474:
2025-02-07 13:47:29,922 - INFO -   Topics: lifecycle, delete, s3, amazon, object
2025-02-07 13:47:29,923 - INFO -   Tokens: 146
2025-02-07 13:47:29,965 - INFO - Created chunk 2475:
2025-02-07 13:47:29,966 - INFO -   Topics: multipart, lifecycle, upload, action, rule
2025-02-07 13:47:29,967 - INFO -   Tokens: 288
2025-02-07 13:47:30,020 - INFO - Created chunk 2476:
2025-02-07 13:47:30,021 - INFO -   Topics: class, upload, region, multipart, storage
2025-02-07 13:47:30,021 - INFO -   Tokens: 371
2025-02-07 13:47:30,078 - INFO - Created chunk 2477:
2025-02-07 13:47:30,078 - INFO -   Topics: s3, aws, amazon, objects, bucket
2025-02-07 13:47:30,079 - INFO -   Tokens: 367
2025-02-07 13:47:30,131 - INFO - Created chunk 2478:
2025-02-07 13:47:30,132 - INFO -   Topics: policies, s3, metrics, amazon, bucket
2025-02-07 13:47:30,132 - INFO -   Tokens: 349
2025-02-07 13:47:30,218 - INFO - Created chu

In [21]:
# Usage:
chunker = SemanticChunker('s3-api.pdf')
#chunker.debug_pdf_content()  # Run this before process_and_save to check content

chunker.process_and_save('NEW-s3-api-chunks.json')


2025-02-07 13:50:01,649 - INFO - Initializing SemanticChunker with PDF: s3-api.pdf
2025-02-07 13:50:02,366 - INFO - Starting PDF processing...
2025-02-07 13:50:02,367 - INFO - Extracting ToC and content from PDF...
2025-02-07 13:51:34,569 - INFO - Extracted 3690305 characters of content
2025-02-07 13:51:34,570 - INFO - Creating semantic chunks...
2025-02-07 13:51:34,580 - INFO - Split content into 3013 paragraphs
2025-02-07 13:51:34,895 - INFO - TF-IDF vectorizer fitted successfully
2025-02-07 13:51:34,927 - INFO - Created chunk 1:
2025-02-07 13:51:34,928 - INFO -   Topics: amazon, service, api, simple, storage
2025-02-07 13:51:34,929 - INFO -   Tokens: 146
2025-02-07 13:51:34,995 - INFO - Created chunk 2:
2025-02-07 13:51:34,995 - INFO -   Topics: using, request, s3, aws, requests
2025-02-07 13:51:34,996 - INFO -   Tokens: 266
2025-02-07 13:51:35,051 - INFO - Created chunk 3:
2025-02-07 13:51:35,051 - INFO -   Topics: signature, examples, requests, using, policy
2025-02-07 13:51:35,05

2025-02-07 13:51:37,138 - INFO -   Tokens: 531
2025-02-07 13:51:37,197 - INFO - Created chunk 45:
2025-02-07 13:51:37,197 - INFO -   Topics: outposts, content, s3, object, type
2025-02-07 13:51:37,198 - INFO -   Tokens: 289
2025-02-07 13:51:37,266 - INFO - Created chunk 46:
2025-02-07 13:51:37,267 - INFO -   Topics: owner, bucket, object, control, objects
2025-02-07 13:51:37,269 - INFO -   Tokens: 382
2025-02-07 13:51:37,356 - INFO - Created chunk 47:
2025-02-07 13:51:37,357 - INFO -   Topics: object, point, access, value, use
2025-02-07 13:51:37,358 - INFO -   Tokens: 497
2025-02-07 13:51:37,429 - INFO - Created chunk 48:
2025-02-07 13:51:37,430 - INFO -   Topics: id, object, version, s3, outposts
2025-02-07 13:51:37,431 - INFO -   Tokens: 397
2025-02-07 13:51:37,520 - INFO - Created chunk 49:
2025-02-07 13:51:37,521 - INFO -   Topics: amz, returns, speciﬁed, tag, object
2025-02-07 13:51:37,522 - INFO -   Tokens: 483
2025-02-07 13:51:37,591 - INFO - Created chunk 50:
2025-02-07 13:51:

2025-02-07 13:51:40,273 - INFO -   Tokens: 351
2025-02-07 13:51:40,324 - INFO - Created chunk 91:
2025-02-07 13:51:40,325 - INFO -   Topics: permissions, owner, access, account, id
2025-02-07 13:51:40,325 - INFO -   Tokens: 338
2025-02-07 13:51:40,372 - INFO - Created chunk 92:
2025-02-07 13:51:40,373 - INFO -   Topics: permissions, access, header, id, account
2025-02-07 13:51:40,373 - INFO -   Tokens: 316
2025-02-07 13:51:40,423 - INFO - Created chunk 93:
2025-02-07 13:51:40,423 - INFO -   Topics: permissions, supported, access, header, id
2025-02-07 13:51:40,424 - INFO -   Tokens: 331
2025-02-07 13:51:40,472 - INFO - Created chunk 94:
2025-02-07 13:51:40,472 - INFO -   Topics: permissions, supported, access, header, id
2025-02-07 13:51:40,473 - INFO -   Tokens: 329
2025-02-07 13:51:40,510 - INFO - Created chunk 95:
2025-02-07 13:51:40,511 - INFO -   Topics: supported, object, amz, directory, buckets
2025-02-07 13:51:40,511 - INFO -   Tokens: 236
2025-02-07 13:51:40,571 - INFO - Creat

2025-02-07 13:51:42,374 - INFO -   Tokens: 124
2025-02-07 13:51:42,414 - INFO - Created chunk 137:
2025-02-07 13:51:42,415 - INFO -   Topics: permissions, bucket, owner, request, conﬁguration
2025-02-07 13:51:42,415 - INFO -   Tokens: 229
2025-02-07 13:51:42,456 - INFO - Created chunk 138:
2025-02-07 13:51:42,457 - INFO -   Topics: id, http, owner, request, body
2025-02-07 13:51:42,458 - INFO -   Tokens: 218
2025-02-07 13:51:42,488 - INFO - Created chunk 139:
2025-02-07 13:51:42,489 - INFO -   Topics: sdk, aws, v3, id, v2
2025-02-07 13:51:42,490 - INFO -   Tokens: 161
2025-02-07 13:51:42,526 - INFO - Created chunk 140:
2025-02-07 13:51:42,527 - INFO -   Topics: owner, bucket, s3, permissions, request
2025-02-07 13:51:42,527 - INFO -   Tokens: 214
2025-02-07 13:51:42,560 - INFO - Created chunk 141:
2025-02-07 13:51:42,561 - INFO -   Topics: http, response, request, body, example
2025-02-07 13:51:42,562 - INFO -   Tokens: 178
2025-02-07 13:51:42,580 - INFO - Created chunk 142:
2025-02-07

2025-02-07 13:51:44,384 - INFO - Created chunk 183:
2025-02-07 13:51:44,385 - INFO -   Topics: sdk, aws, v3, v2, ruby
2025-02-07 13:51:44,386 - INFO -   Tokens: 52
2025-02-07 13:51:44,422 - INFO - Created chunk 184:
2025-02-07 13:51:44,423 - INFO -   Topics: permissions, bucket, s3, amazon, delete
2025-02-07 13:51:44,423 - INFO -   Tokens: 167
2025-02-07 13:51:44,459 - INFO - Created chunk 185:
2025-02-07 13:51:44,460 - INFO -   Topics: sdk, aws, owner, body, http
2025-02-07 13:51:44,460 - INFO -   Tokens: 190
2025-02-07 13:51:44,507 - INFO - Created chunk 186:
2025-02-07 13:51:44,508 - INFO -   Topics: bucket, permissions, s3, owner, operation
2025-02-07 13:51:44,508 - INFO -   Tokens: 293
2025-02-07 13:51:44,548 - INFO - Created chunk 187:
2025-02-07 13:51:44,549 - INFO -   Topics: request, bucket, owner, status, buckets
2025-02-07 13:51:44,549 - INFO -   Tokens: 257
2025-02-07 13:51:44,582 - INFO - Created chunk 188:
2025-02-07 13:51:44,583 - INFO -   Topics: response, returns, vali

...

2025-02-07 13:53:35,259 - INFO -   Topics: type, default, content, response, access
2025-02-07 13:53:35,260 - INFO -   Tokens: 238
2025-02-07 13:53:35,305 - INFO - Created chunk 2728:
2025-02-07 13:53:35,306 - INFO -   Topics: object, encryption, objects, server, kms
2025-02-07 13:53:35,307 - INFO -   Tokens: 295
2025-02-07 13:53:35,346 - INFO - Created chunk 2729:
2025-02-07 13:53:35,347 - INFO -   Topics: amz, default, server, used, aws
2025-02-07 13:53:35,347 - INFO -   Tokens: 256
2025-02-07 13:53:35,368 - INFO - Created chunk 2730:
2025-02-07 13:53:35,369 - INFO -   Topics: object, string, default, bucket, version
2025-02-07 13:53:35,370 - INFO -   Tokens: 106
2025-02-07 13:53:35,412 - INFO - Created chunk 2731:
2025-02-07 13:53:35,413 - INFO -   Topics: error, list, s3, http, amazon
2025-02-07 13:53:35,413 - INFO -   Tokens: 277
2025-02-07 13:53:35,454 - INFO - Created chunk 2732:
2025-02-07 13:53:35,455 - INFO -   Topics: error, code, response, type, xml
2025-02-07 13:53:35,455 

2025-02-07 13:53:37,087 - INFO - Created chunk 2773:
2025-02-07 13:53:37,088 - INFO -   Topics: error, outposts, supported, code, list
2025-02-07 13:53:37,088 - INFO -   Tokens: 230
2025-02-07 13:53:37,134 - INFO - Created chunk 2774:
2025-02-07 13:53:37,135 - INFO -   Topics: supported, error, account, code, list
2025-02-07 13:53:37,135 - INFO -   Tokens: 214
2025-02-07 13:53:37,176 - INFO - Created chunk 2775:
2025-02-07 13:53:37,177 - INFO -   Topics: supported, error, conﬁguration, code, storage
2025-02-07 13:53:37,177 - INFO -   Tokens: 152
2025-02-07 13:53:37,232 - INFO - Created chunk 2776:
2025-02-07 13:53:37,232 - INFO -   Topics: supported, conﬁguration, error, arn, code
2025-02-07 13:53:37,233 - INFO -   Tokens: 207
2025-02-07 13:53:37,282 - INFO - Created chunk 2777:
2025-02-07 13:53:37,283 - INFO -   Topics: supported, conﬁguration, arn, valid, default
2025-02-07 13:53:37,283 - INFO -   Tokens: 254
2025-02-07 13:53:37,340 - INFO - Created chunk 2778:
2025-02-07 13:53:37,34

2025-02-07 13:53:39,073 - INFO -   Tokens: 264
2025-02-07 13:53:39,120 - INFO - Created chunk 2819:
2025-02-07 13:53:39,121 - INFO -   Topics: upload, encryption, data, objects, server
2025-02-07 13:53:39,121 - INFO -   Tokens: 307
2025-02-07 13:53:39,161 - INFO - Created chunk 2820:
2025-02-07 13:53:39,161 - INFO -   Topics: error, object, response, returns, amazon
2025-02-07 13:53:39,162 - INFO -   Tokens: 238
2025-02-07 13:53:39,217 - INFO - Created chunk 2821:
2025-02-07 13:53:39,217 - INFO -   Topics: upload, objects, bucket, information, amazon
2025-02-07 13:53:39,218 - INFO -   Tokens: 344
2025-02-07 13:53:39,275 - INFO - Created chunk 2822:
2025-02-07 13:53:39,276 - INFO -   Topics: object, region, new, objects, s3
2025-02-07 13:53:39,277 - INFO -   Tokens: 352
2025-02-07 13:53:39,304 - INFO - Created chunk 2823:
2025-02-07 13:53:39,305 - INFO -   Topics: api, amazon, signature, version, storage
2025-02-07 13:53:39,306 - INFO -   Tokens: 154
2025-02-07 13:53:39,361 - INFO - Cre

2025-02-07 13:53:41,258 - INFO -   Topics: default, tag, object, returns, type
2025-02-07 13:53:41,259 - INFO -   Tokens: 184
2025-02-07 13:53:41,289 - INFO - Created chunk 2865:
2025-02-07 13:53:41,290 - INFO -   Topics: response, returns, object, api, type
2025-02-07 13:53:41,291 - INFO -   Tokens: 172
2025-02-07 13:53:41,333 - INFO - Created chunk 2866:
2025-02-07 13:53:41,334 - INFO -   Topics: amazonaws, com, signature, object, http
2025-02-07 13:53:41,335 - INFO -   Tokens: 248
2025-02-07 13:53:41,379 - INFO - Created chunk 2867:
2025-02-07 13:53:41,380 - INFO -   Topics: value, api, use, signature, returns
2025-02-07 13:53:41,380 - INFO -   Tokens: 277
2025-02-07 13:53:41,421 - INFO - Created chunk 2868:
2025-02-07 13:53:41,422 - INFO -   Topics: value, key, signature, used, status
2025-02-07 13:53:41,423 - INFO -   Tokens: 252
2025-02-07 13:53:41,479 - INFO - Created chunk 2869:
2025-02-07 13:53:41,480 - INFO -   Topics: object, data, objects, key, xml
2025-02-07 13:53:41,480 -

2025-02-07 13:53:43,541 - INFO -   Tokens: 173
2025-02-07 13:53:43,576 - INFO - Created chunk 2910:
2025-02-07 13:53:43,576 - INFO -   Topics: status, value, set, code, returns
2025-02-07 13:53:43,577 - INFO -   Tokens: 210
2025-02-07 13:53:43,614 - INFO - Created chunk 2911:
2025-02-07 13:53:43,615 - INFO -   Topics: signature, policy, content, access, required
2025-02-07 13:53:43,616 - INFO -   Tokens: 223
2025-02-07 13:53:43,667 - INFO - Created chunk 2912:
2025-02-07 13:53:43,668 - INFO -   Topics: policy, signature, date, user, example
2025-02-07 13:53:43,668 - INFO -   Tokens: 281
2025-02-07 13:53:43,720 - INFO - Created chunk 2913:
2025-02-07 13:53:43,720 - INFO -   Topics: content, policy, key, user, value
2025-02-07 13:53:43,721 - INFO -   Tokens: 330
2025-02-07 13:53:43,763 - INFO - Created chunk 2914:
2025-02-07 13:53:43,764 - INFO -   Topics: public, speciﬁed, set, example, signature
2025-02-07 13:53:43,765 - INFO -   Tokens: 266
2025-02-07 13:53:43,801 - INFO - Created chu

In [22]:
# Usage:
chunker = SemanticChunker('s3-outposts.pdf')
#chunker.debug_pdf_content()  # Run this before process_and_save to check content

chunker.process_and_save('NEW-s3-outposts-chunks.json')


2025-02-07 13:53:45,909 - INFO - Initializing SemanticChunker with PDF: s3-outposts.pdf
2025-02-07 13:53:46,546 - INFO - Starting PDF processing...
2025-02-07 13:53:46,547 - INFO - Extracting ToC and content from PDF...
2025-02-07 13:54:01,841 - INFO - Extracted 390859 characters of content
2025-02-07 13:54:01,842 - INFO - Creating semantic chunks...
2025-02-07 13:54:01,843 - INFO - Split content into 217 paragraphs
2025-02-07 13:54:01,890 - INFO - TF-IDF vectorizer fitted successfully
2025-02-07 13:54:01,921 - INFO - Created chunk 1:
2025-02-07 13:54:01,921 - INFO -   Topics: amazon, services, guide, s3, user
2025-02-07 13:54:01,922 - INFO -   Tokens: 146
2025-02-07 13:54:01,980 - INFO - Created chunk 2:
2025-02-07 13:54:01,981 - INFO -   Topics: s3, outposts, storage, aws, console
2025-02-07 13:54:01,981 - INFO -   Tokens: 241
2025-02-07 13:54:02,040 - INFO - Created chunk 3:
2025-02-07 13:54:02,041 - INFO -   Topics: access, bucket, policy, point, creating
2025-02-07 13:54:02,041 - 

2025-02-07 13:54:05,664 - INFO - Created chunk 44:
2025-02-07 13:54:05,665 - INFO -   Topics: bucket, policy, s3, outposts, permissions
2025-02-07 13:54:05,666 - INFO -   Tokens: 376
2025-02-07 13:54:05,811 - INFO - Created chunk 45:
2025-02-07 13:54:05,812 - INFO -   Topics: ip, policy, bucket, access, outposts
2025-02-07 13:54:05,813 - INFO -   Tokens: 413
2025-02-07 13:54:05,901 - INFO - Created chunk 46:
2025-02-07 13:54:05,902 - INFO -   Topics: ipv6, example, policy, ip, outposts
2025-02-07 13:54:05,903 - INFO -   Tokens: 354
2025-02-07 13:54:06,021 - INFO - Created chunk 47:
2025-02-07 13:54:06,022 - INFO -   Topics: aws, ip, ipv6, bucket, example
2025-02-07 13:54:06,023 - INFO -   Tokens: 339
2025-02-07 13:54:06,124 - INFO - Created chunk 48:
2025-02-07 13:54:06,126 - INFO -   Topics: buckets, s3, aws, outposts, list
2025-02-07 13:54:06,127 - INFO -   Tokens: 384
2025-02-07 13:54:06,246 - INFO - Created chunk 49:
2025-02-07 13:54:06,247 - INFO -   Topics: aws, outposts, s3, dat

2025-02-07 13:54:09,445 - INFO -   Topics: import, upload, object, amazonaws, com
2025-02-07 13:54:09,445 - INFO -   Tokens: 327
2025-02-07 13:54:09,499 - INFO - Created chunk 91:
2025-02-07 13:54:09,499 - INFO -   Topics: new, object, upload, list, s3
2025-02-07 13:54:09,500 - INFO -   Tokens: 286
2025-02-07 13:54:09,553 - INFO - Created chunk 92:
2025-02-07 13:54:09,554 - INFO -   Topics: import, response, list, amazonaws, string
2025-02-07 13:54:09,554 - INFO -   Tokens: 340
2025-02-07 13:54:09,598 - INFO - Created chunk 93:
2025-02-07 13:54:09,598 - INFO -   Topics: new, object, upload, list, using
2025-02-07 13:54:09,599 - INFO -   Tokens: 285
2025-02-07 13:54:09,646 - INFO - Created chunk 94:
2025-02-07 13:54:09,647 - INFO -   Topics: list, response, import, amazonaws, com
2025-02-07 13:54:09,648 - INFO -   Tokens: 320
2025-02-07 13:54:09,698 - INFO - Created chunk 95:
2025-02-07 13:54:09,699 - INFO -   Topics: string, response, upload, list, public
2025-02-07 13:54:09,699 - INFO

2025-02-07 13:54:11,799 - INFO - Created chunk 136:
2025-02-07 13:54:11,800 - INFO -   Topics: presigned, examples, bucket, s3, outposts
2025-02-07 13:54:11,801 - INFO -   Tokens: 330
2025-02-07 13:54:11,849 - INFO - Created chunk 137:
2025-02-07 13:54:11,849 - INFO -   Topics: presigned, requests, policy, url, bucket
2025-02-07 13:54:11,850 - INFO -   Tokens: 343
2025-02-07 13:54:11,906 - INFO - Created chunk 138:
2025-02-07 13:54:11,907 - INFO -   Topics: aws, policies, permissions, policy, use
2025-02-07 13:54:11,908 - INFO -   Tokens: 366
2025-02-07 13:54:11,955 - INFO - Created chunk 139:
2025-02-07 13:54:11,956 - INFO -   Topics: linked, service, outposts, permissions, role
2025-02-07 13:54:11,956 - INFO -   Tokens: 323
2025-02-07 13:54:12,002 - INFO - Created chunk 140:
2025-02-07 13:54:12,003 - INFO -   Topics: linked, role, service, services, permissions
2025-02-07 13:54:12,003 - INFO -   Tokens: 323
2025-02-07 13:54:12,039 - INFO - Created chunk 141:
2025-02-07 13:54:12,040 -

2025-02-07 13:54:14,321 - INFO -   Topics: replication, object, source, outposts, s3
2025-02-07 13:54:14,321 - INFO -   Tokens: 408
2025-02-07 13:54:14,402 - INFO - Created chunk 181:
2025-02-07 13:54:14,403 - INFO -   Topics: replication, bucket, replicate, source, destination
2025-02-07 13:54:14,404 - INFO -   Tokens: 530
2025-02-07 13:54:14,481 - INFO - Created chunk 182:
2025-02-07 13:54:14,482 - INFO -   Topics: replication, events, outposts, s3, destination
2025-02-07 13:54:14,482 - INFO -   Tokens: 345
2025-02-07 13:54:14,532 - INFO - Created chunk 183:
2025-02-07 13:54:14,532 - INFO -   Topics: replication, destination, outposts, s3, bucket
2025-02-07 13:54:14,534 - INFO -   Tokens: 274
2025-02-07 13:54:14,571 - INFO - Created chunk 184:
2025-02-07 13:54:14,572 - INFO -   Topics: destination, bucket, replicate, s3, outposts
2025-02-07 13:54:14,573 - INFO -   Tokens: 254
2025-02-07 13:54:14,611 - INFO - Created chunk 185:
2025-02-07 13:54:14,611 - INFO -   Topics: replication, s

In [23]:
# Initialize and run the chunker
chunker = SemanticChunker('glacier-dg.pdf')
chunker.process_and_save('NEW-glacier-dg-chunks.json')

2025-02-07 13:54:16,095 - INFO - Initializing SemanticChunker with PDF: glacier-dg.pdf
2025-02-07 13:54:16,925 - INFO - Starting PDF processing...
2025-02-07 13:54:16,926 - INFO - Extracting ToC and content from PDF...
2025-02-07 13:54:49,169 - INFO - Extracted 865448 characters of content
2025-02-07 13:54:49,170 - INFO - Creating semantic chunks...
2025-02-07 13:54:49,171 - INFO - Split content into 558 paragraphs
2025-02-07 13:54:49,247 - INFO - TF-IDF vectorizer fitted successfully
2025-02-07 13:54:49,275 - INFO - Created chunk 1:
2025-02-07 13:54:49,275 - INFO -   Topics: amazon, services, s3, developer, glacier
2025-02-07 13:54:49,276 - INFO -   Tokens: 143
2025-02-07 13:54:49,321 - INFO - Created chunk 2:
2025-02-07 13:54:49,322 - INFO -   Topics: archive, vault, using, download, delete
2025-02-07 13:54:49,323 - INFO -   Tokens: 268
2025-02-07 13:54:49,372 - INFO - Created chunk 3:
2025-02-07 13:54:49,373 - INFO -   Topics: vault, using, inventory, downloading, net
2025-02-07 13:

2025-02-07 13:54:51,526 - INFO -   Topics: aws, vault, s3, glacier, cli
2025-02-07 13:54:51,526 - INFO -   Tokens: 319
2025-02-07 13:54:51,581 - INFO - Created chunk 45:
2025-02-07 13:54:51,582 - INFO -   Topics: vault, inventory, s3, glacier, operations
2025-02-07 13:54:51,582 - INFO -   Tokens: 412
2025-02-07 13:54:51,645 - INFO - Created chunk 46:
2025-02-07 13:54:51,646 - INFO -   Topics: inventory, vault, job, notiﬁcation, sns
2025-02-07 13:54:51,646 - INFO -   Tokens: 505
2025-02-07 13:54:51,699 - INFO - Created chunk 47:
2025-02-07 13:54:51,700 - INFO -   Topics: vault, aws, using, java, provide
2025-02-07 13:54:51,700 - INFO -   Tokens: 411
2025-02-07 13:54:51,744 - INFO - Created chunk 48:
2025-02-07 13:54:51,744 - INFO -   Topics: import, java, amazonaws, vault, services
2025-02-07 13:54:51,745 - INFO -   Tokens: 313
2025-02-07 13:54:51,778 - INFO - Created chunk 49:
2025-02-07 13:54:51,779 - INFO -   Topics: client, vaultname, import, services, amazonaws
2025-02-07 13:54:51,

2025-02-07 13:54:53,690 - INFO -   Topics: notiﬁcation, vault, net, provide, using
2025-02-07 13:54:53,690 - INFO -   Tokens: 373
2025-02-07 13:54:53,727 - INFO - Created chunk 91:
2025-02-07 13:54:53,728 - INFO -   Topics: using, static, console, amazon, sns
2025-02-07 13:54:53,728 - INFO -   Tokens: 280
2025-02-07 13:54:53,764 - INFO - Created chunk 92:
2025-02-07 13:54:53,764 - INFO -   Topics: console, vaultname, new, response, request
2025-02-07 13:54:53,765 - INFO -   Tokens: 266
2025-02-07 13:54:53,810 - INFO - Created chunk 93:
2025-02-07 13:54:53,811 - INFO -   Topics: notiﬁcation, vault, topic, sns, console
2025-02-07 13:54:53,811 - INFO -   Tokens: 358
2025-02-07 13:54:53,847 - INFO - Created chunk 94:
2025-02-07 13:54:53,847 - INFO -   Topics: topic, sns, arn, console, amazon
2025-02-07 13:54:53,848 - INFO -   Tokens: 256
2025-02-07 13:54:53,890 - INFO - Created chunk 95:
2025-02-07 13:54:53,890 - INFO -   Topics: notiﬁcation, cli, command, aws, vault
2025-02-07 13:54:53,89

2025-02-07 13:54:55,835 - INFO -   Topics: upload, range, id, multipart, hash
2025-02-07 13:54:55,836 - INFO -   Tokens: 346
2025-02-07 13:54:55,893 - INFO - Created chunk 137:
2025-02-07 13:54:55,894 - INFO -   Topics: level, upload, archives, parts, uploading
2025-02-07 13:54:55,895 - INFO -   Tokens: 449
2025-02-07 13:54:55,953 - INFO - Created chunk 138:
2025-02-07 13:54:55,954 - INFO -   Topics: upload, parts, archive, java, sdk
2025-02-07 13:54:55,954 - INFO -   Tokens: 434
2025-02-07 13:54:55,993 - INFO - Created chunk 139:
2025-02-07 13:54:55,993 - INFO -   Topics: import, amazonaws, com, services, java
2025-02-07 13:54:55,994 - INFO -   Tokens: 265
2025-02-07 13:54:56,032 - INFO - Created chunk 140:
2025-02-07 13:54:56,032 - INFO -   Topics: new, string, client, uploading, static
2025-02-07 13:54:56,033 - INFO -   Tokens: 265
2025-02-07 13:54:56,069 - INFO - Created chunk 141:
2025-02-07 13:54:56,070 - INFO -   Topics: string, new, client, vaultname, parts
2025-02-07 13:54:56,

2025-02-07 13:54:57,904 - INFO -   Tokens: 306
2025-02-07 13:54:57,936 - INFO - Created chunk 182:
2025-02-07 13:54:57,937 - INFO -   Topics: new, client, string, archive, job
2025-02-07 13:54:57,937 - INFO -   Tokens: 256
2025-02-07 13:54:57,979 - INFO - Created chunk 183:
2025-02-07 13:54:57,980 - INFO -   Topics: string, console, new, job, downloading
2025-02-07 13:54:57,980 - INFO -   Tokens: 321
2025-02-07 13:54:58,013 - INFO - Created chunk 184:
2025-02-07 13:54:58,013 - INFO -   Topics: length, output, archive, downloading, job
2025-02-07 13:54:58,014 - INFO -   Tokens: 250
2025-02-07 13:54:58,046 - INFO - Created chunk 185:
2025-02-07 13:54:58,047 - INFO -   Topics: aws, cli, command, list, using
2025-02-07 13:54:58,048 - INFO -   Tokens: 241
2025-02-07 13:54:58,087 - INFO - Created chunk 186:
2025-02-07 13:54:58,088 - INFO -   Topics: job, archive, inventory, download, command
2025-02-07 13:54:58,088 - INFO -   Tokens: 313
2025-02-07 13:54:58,140 - INFO - Created chunk 187:
20

...

2025-02-07 13:55:06,503 - INFO -   Tokens: 234
2025-02-07 13:55:06,572 - INFO - Created chunk 367:
2025-02-07 13:55:06,572 - INFO -   Topics: range, tree, hash, download, job
2025-02-07 13:55:06,573 - INFO -   Tokens: 506
2025-02-07 13:55:06,608 - INFO - Created chunk 368:
2025-02-07 13:55:06,609 - INFO -   Topics: tree, hash, range, data, specify
2025-02-07 13:55:06,610 - INFO -   Tokens: 242
2025-02-07 13:55:06,702 - INFO - Created chunk 369:
2025-02-07 13:55:06,703 - INFO -   Topics: tree, hash, range, data, archive
2025-02-07 13:55:06,704 - INFO -   Tokens: 453
2025-02-07 13:55:06,758 - INFO - Created chunk 370:
2025-02-07 13:55:06,759 - INFO -   Topics: request, tags, service, access, description
2025-02-07 13:55:06,760 - INFO -   Tokens: 203
2025-02-07 13:55:06,816 - INFO - Created chunk 371:
2025-02-07 13:55:06,817 - INFO -   Topics: retrieval, data, policies, s3, glacier
2025-02-07 13:55:06,818 - INFO -   Tokens: 232
2025-02-07 13:55:06,871 - INFO - Created chunk 372:
2025-02-0

2025-02-07 13:55:08,796 - INFO -   Topics: request, headers, id, lock, account
2025-02-07 13:55:08,797 - INFO -   Tokens: 262
2025-02-07 13:55:08,829 - INFO - Created chunk 413:
2025-02-07 13:55:08,829 - INFO -   Topics: response, lock, headers, common, policy
2025-02-07 13:55:08,830 - INFO -   Tokens: 203
2025-02-07 13:55:08,883 - INFO - Created chunk 414:
2025-02-07 13:55:08,884 - INFO -   Topics: lock, amz, policy, http, glacier
2025-02-07 13:55:08,884 - INFO -   Tokens: 267
2025-02-07 13:55:08,956 - INFO - Created chunk 415:
2025-02-07 13:55:08,957 - INFO -   Topics: tags, id, account, request, vault
2025-02-07 13:55:08,959 - INFO -   Tokens: 279
2025-02-07 13:55:09,024 - INFO - Created chunk 416:
2025-02-07 13:55:09,024 - INFO -   Topics: headers, common, response, tags, string
2025-02-07 13:55:09,025 - INFO -   Tokens: 222
2025-02-07 13:55:09,087 - INFO - Created chunk 417:
2025-02-07 13:55:09,088 - INFO -   Topics: tags, amz, example, list, glacier
2025-02-07 13:55:09,088 - INFO

2025-02-07 13:55:11,020 - INFO -   Tokens: 358
2025-02-07 13:55:11,067 - INFO - Created chunk 458:
2025-02-07 13:55:11,068 - INFO -   Topics: parts, headers, response, common, value
2025-02-07 13:55:11,068 - INFO -   Tokens: 218
2025-02-07 13:55:11,110 - INFO - Created chunk 459:
2025-02-07 13:55:11,111 - INFO -   Topics: string, response, parts, headers, multipart
2025-02-07 13:55:11,112 - INFO -   Tokens: 258
2025-02-07 13:55:11,146 - INFO - Created chunk 460:
2025-02-07 13:55:11,147 - INFO -   Topics: type, string, value, parts, upload
2025-02-07 13:55:11,148 - INFO -   Tokens: 199
2025-02-07 13:55:11,197 - INFO - Created chunk 461:
2025-02-07 13:55:11,198 - INFO -   Topics: parts, multipart, upload, amz, example
2025-02-07 13:55:11,199 - INFO -   Tokens: 303
2025-02-07 13:55:11,256 - INFO - Created chunk 462:
2025-02-07 13:55:11,257 - INFO -   Topics: multipart, amz, list, example, parts
2025-02-07 13:55:11,258 - INFO -   Tokens: 374
2025-02-07 13:55:11,305 - INFO - Created chunk 4

2025-02-07 13:55:13,402 - INFO -   Tokens: 293
2025-02-07 13:55:13,438 - INFO - Created chunk 504:
2025-02-07 13:55:13,439 - INFO -   Topics: job, id, request, body, amz
2025-02-07 13:55:13,439 - INFO -   Tokens: 216
2025-02-07 13:55:13,473 - INFO - Created chunk 505:
2025-02-07 13:55:13,474 - INFO -   Topics: job, amz, body, description, type
2025-02-07 13:55:13,474 - INFO -   Tokens: 196
2025-02-07 13:55:13,522 - INFO - Created chunk 506:
2025-02-07 13:55:13,523 - INFO -   Topics: west, amz, archive, description, example
2025-02-07 13:55:13,524 - INFO -   Tokens: 303
2025-02-07 13:55:13,570 - INFO - Created chunk 507:
2025-02-07 13:55:13,570 - INFO -   Topics: amz, inventory, job, jobs, west
2025-02-07 13:55:13,571 - INFO -   Tokens: 292
2025-02-07 13:55:13,615 - INFO - Created chunk 508:
2025-02-07 13:55:13,616 - INFO -   Topics: inventory, retrieval, job, request, description
2025-02-07 13:55:13,617 - INFO -   Tokens: 300
2025-02-07 13:55:13,663 - INFO - Created chunk 509:
2025-02-

2025-02-07 13:55:15,343 - INFO -   Tokens: 256
2025-02-07 13:55:15,385 - INFO - Created chunk 550:
2025-02-07 13:55:15,385 - INFO -   Topics: amz, west, glacier, http, date
2025-02-07 13:55:15,386 - INFO -   Tokens: 255
2025-02-07 13:55:15,425 - INFO - Created chunk 551:
2025-02-07 13:55:15,426 - INFO -   Topics: policy, amz, http, retrieval, data
2025-02-07 13:55:15,426 - INFO -   Tokens: 226
2025-02-07 13:55:15,438 - INFO - Created chunk 552:
2025-02-07 13:55:15,439 - INFO -   Topics: jobs, post, policy, retrieval, data
2025-02-07 13:55:15,440 - INFO -   Tokens: 25
2025-02-07 13:55:15,488 - INFO - Created chunk 553:
2025-02-07 13:55:15,489 - INFO -   Topics: s3, glacier, amazon, retrieval, guide
2025-02-07 13:55:15,490 - INFO -   Tokens: 255
2025-02-07 13:55:15,558 - INFO - Created chunk 554:
2025-02-07 13:55:15,559 - INFO -   Topics: policies, data, lock, retrieval, access
2025-02-07 13:55:15,559 - INFO -   Tokens: 371
2025-02-07 13:55:15,617 - INFO - Created chunk 555:
2025-02-07 1